In [1]:
import pandas as pd 
import numpy as np
import public_timeseries_testing_util as optiver2023
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, unpack_sequence, unpad_sequence
import torch
from tqdm.notebook import trange,tqdm
import torch.nn as nn 
import torch.optim as optim
import wandb
import torch_classes
from model_saver import model_saver_wandb as model_saver
import training_testing

In [2]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
train = pd.read_csv('train.csv')
train.head()
train.date_id.value_counts()

date_id
480    11000
353    11000
363    11000
362    11000
360    11000
       ...  
4      10560
2      10505
1      10505
3      10505
0      10505
Name: count, Length: 481, dtype: int64

In [5]:
import importlib

In [6]:
importlib.reload(torch_classes)
trading_data = torch_classes.TradingData(train)
hidden_size = 64
trading_data.generate_batches()

 10%|▉         | 19/200 [00:08<01:16,  2.37it/s]

Missing Targets for day=438,for stock_id=19, Excluding


 50%|█████     | 101/200 [00:41<00:35,  2.79it/s]

Missing Targets for day=328,for stock_id=101, Excluding


 66%|██████▌   | 131/200 [00:55<00:46,  1.50it/s]

...

 79%|███████▉  | 158/200 [01:05<00:15,  2.78it/s]

Missing Targets for day=388,for stock_id=158, Excluding


100%|██████████| 200/200 [01:25<00:00,  2.35it/s]


Length of train: 385, Length of test 96


100%|██████████| 95/95 [00:00<00:00, 11866.67it/s]


In [7]:
trading_data.packed_val_x[0]

PackedSequence(data=tensor([[ 0.0000e+00,  2.5554e+06, -1.0000e+00,  ...,  1.0000e+00,
          2.4081e+03,  1.0000e+00],
        [ 0.0000e+00,  2.7470e+05,  1.0000e+00,  ...,  1.0000e+00,
          1.7408e+02,  1.0000e+00],
        [ 0.0000e+00,  4.1553e+05,  1.0000e+00,  ...,  1.0003e+00,
          2.2485e+04,  1.0000e+00],
        ...,
        [ 5.4000e+02,  0.0000e+00,  0.0000e+00,  ...,  9.9962e-01,
          1.0968e+02,  9.9962e-01],
        [ 5.4000e+02,  0.0000e+00,  0.0000e+00,  ...,  9.9673e-01,
          2.1029e+05,  9.9664e-01],
        [ 5.4000e+02,  0.0000e+00,  0.0000e+00,  ...,  1.0019e+00,
          8.6312e+04,  1.0017e+00]], device='cuda:0'), batch_sizes=tensor([200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,
        200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200]), sorted_in

In [8]:
importlib.reload(torch_classes)

<module 'torch_classes' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\torch_classes.py'>

In [9]:
model = torch_classes.GRUNetV2(12,64).to(device='cuda:0')

c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [10]:
output,hidden = model(trading_data.packed_x[0],p1=True)

In [11]:
stocks = [trading_data.stocksDict[x] for x in trading_data.stock_batches[0]]

In [12]:
len(stocks)

191

In [13]:
len(output[0][0])

64

In [14]:
len(output[1])

55

In [15]:
[setattr(obj, 'hidden_all', val) for obj, val in zip(stocks,output)]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [16]:
len(stocks[0].hidden_all)

55

In [17]:
# torch.stack([stocks[x].hidden_out ])

In [18]:
# stocks[0].hidden_out

In [19]:
# for i in range(0,199):
    # print(i)

In [20]:
stocks_hidden,targets = trading_data.fetch_daily_data(day=0)

In [21]:
stocks_hidden[0].shape

torch.Size([55, 64])

In [22]:
X = torch.cat(stocks_hidden,dim=-1)
# Y = 

In [23]:
output = model(X)

In [24]:
# model()

In [25]:
targets[199].shape

torch.Size([55])

In [26]:
Y = torch.stack(targets).transpose(0,1).to('cuda:0')

In [27]:
model(X)[0]

tensor([ 6.4300e-02, -4.0778e-02,  1.5305e-02,  3.1057e-02, -1.7071e-02,
        -4.4198e-04, -3.7108e-02,  8.5022e-02,  5.6072e-05, -7.7676e-03,
        -1.3965e-02, -7.6019e-02,  4.2677e-02,  6.0845e-02, -8.0050e-02,
         4.5991e-02, -2.9843e-02, -4.3675e-02,  1.0126e-02,  6.3449e-02,
        -9.2830e-02, -1.2253e-02, -1.6795e-02,  9.9773e-02, -3.3508e-02,
        -6.8902e-02, -5.6889e-02, -2.7819e-02,  3.8781e-03, -3.1606e-02,
         8.1263e-02, -6.8777e-02,  3.6871e-02, -3.8080e-02,  2.4221e-02,
         1.4312e-02, -4.3721e-02, -5.1106e-02,  6.8797e-02, -4.3544e-02,
         7.1691e-02, -6.3958e-02,  4.3330e-02,  1.5699e-02, -3.6409e-02,
         5.8562e-02, -2.8182e-02,  2.4726e-02,  1.1522e-02,  2.2974e-02,
        -9.4520e-04,  3.5101e-02, -3.3950e-02,  6.8962e-02,  2.0669e-02,
         6.0671e-02, -2.5218e-02, -6.0350e-02, -3.3994e-02, -3.0336e-02,
         9.9465e-03,  9.3753e-03,  3.5077e-02,  8.8010e-02,  6.3564e-02,
        -2.8295e-02, -8.8596e-03,  5.0479e-02,  3.6

In [28]:
reg_L1 = nn.L1Loss()

In [29]:
reg_L1(output, Y)

tensor(5.3359, device='cuda:0', grad_fn=<MeanBackward0>)

In [30]:
Y[0]

tensor([ -3.0297,  -5.5200,  -8.3899,  -4.0102,  -7.3498,   6.7794,  -2.4998,
         -1.9598,  -5.9700,   7.9703,   5.3501,   2.5594,   8.3995, -10.7503,
          2.3198,   1.7202,  -8.1700,  -3.5900, -11.4399,   4.2999,  -4.1801,
        -11.5401,   8.9598,   2.4402,   4.4894,  -2.5600,  -0.3499,  -0.0602,
         -1.6898,   4.6396,   4.4096,  -1.3900,   7.8106,   4.4203,  -7.7599,
          0.1895,   3.5405,  -0.4297,   7.8297,  -4.6998,  -1.3798,   4.9603,
        -10.6698,  -3.3402,   3.1400,  -1.4400,  17.1494,  10.1900,  -7.7599,
          9.2602,   4.2605,   6.7103,   1.0002,  -6.3199,  -3.6699,  -1.2100,
         10.4797,   6.8295,  -1.2797,  -3.6001,  -1.1098,   3.1304,  13.7997,
          2.0301, -22.7302,  12.6195,  -5.0402, -11.9698,   4.0805,   0.0000,
          9.6703, -17.8599,  -5.5802,  -2.5600,   3.3605,  -6.6799,  -1.9801,
          0.2694,   0.0000,   0.0000,  -9.6399,  16.6905,   6.5303,  -1.0800,
          6.3896, -18.8702,  30.5200,   6.9702,  -5.0598,  -2.56

In [31]:
def model_pipeline(trading_df:torch_classes.TradingData, config:dict):
    with wandb.init(project="Optviver", config=config,save_code=True):
        wandb.define_metric("val_epoch_loss_l1", summary="min")
        wandb.define_metric("epoch_l1_loss", summary="min")
        model = torch_classes.GRUNetV2(12,hidden_size).to('cuda:0')
        config = wandb.config
        optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
        trading_df.reset_hidden(config['hidden_size'])
        criterion = nn.SmoothL1Loss()
        
        training_testing.train_model(trading_df,model,config,optimizer,criterion)

    return(model)



In [32]:
importlib.reload(training_testing)

<module 'training_testing' from 'c:\\Users\\Nick\\Documents\\GitHub\\OptiverKaggle\\training_testing.py'>

In [33]:
config_static = {'learning_rate':0.00002, 'hidden_size':64}

In [34]:
model = model_pipeline(trading_data, config_static)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: nickojelly. Use `wandb login --relogin` to force relogin


  0%|          | 0/10000 [00:00<?, ?it/s]